In [1]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer
import torch
from utils import util
import json

# Load models
# checkpoint_path = "/Users/alvinchen/Documents/GitHub/brainteaser-data/testing/models/bert-BASED-uncased/"
checkpoint_path = "models/bert-large-and-in-charge-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForMultipleChoice.from_pretrained(checkpoint_path)

In [2]:
# Load datasets
with open("../data/SP/SP_test_clean.json") as f:
    data = f.read()
data = json.loads(data)

In [3]:
# Evaluate dev set
def predict(model, tokenizer, dataset):
    preds = []
    model.eval()
    for sample in dataset:
        inputs = [[sample["question"], answer] for answer in sample["choice_list"]]
        tokens = tokenizer(inputs, return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0)
        outputs = model(**{k: v.unsqueeze(0) for k, v in tokens.items()}, labels=labels)
        logits = outputs.logits
        pred = logits.argmax().item()
        preds.append(pred)
    return preds

preds = predict(model, tokenizer, data)

In [4]:
print(preds)

[0, 2, 2, 1, 3, 0, 0, 1, 3, 1, 1, 0, 2, 0, 0, 1, 0, 2, 2, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 0, 1, 0, 1, 1, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 3, 1, 2, 0, 2, 1, 1, 1, 0, 1, 1, 1, 3, 3, 2, 1, 2, 0, 3, 0, 0, 2, 1, 2, 3, 2, 1, 0, 1, 2, 3, 0, 2, 1, 3, 2, 3, 2, 3, 0, 3, 0, 2, 3, 1, 2, 0, 2, 1, 1, 0, 2, 2, 2, 3, 2, 3, 3, 1, 3, 1, 2, 2, 3, 2, 1, 1, 2]


In [5]:
with open("../submission/answer_sen.txt", "w") as f:
    for pred in preds:
        f.write(str(pred) + "\n")

120
